In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
dosave = True
plotqual = "plane0_"
tqual = ""
savedir = "plots_6_21_23/%s" % plotqual
plt.rcParams.update({'font.size': 14})
plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]

datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

filedirs = [
#     "/pnfs/sbn/persistent/users/gputnam/calib-data/RunA/DPI2023/",
#     "/pnfs/sbn/persistent/users/gputnam/calib-data/RunA/DPFNAL2022/",
#     "/pnfs/sbn/persistent/users/gputnam/calib-data/Run1/",
    "/pnfs/sbn/persistent/users/gputnam/calib-data/Run2/"
]

isMC = False

In [ ]:
files = sum([[filedir + f for f in os.listdir(filedir) if f.endswith(".df") and f.startswith("etau") and
         "C1" not in f and "old" not in f and "S12" in f and "P1" not in f and "P0" in f] for filedir in filedirs], [])

In [ ]:
files

In [ ]:
import gc
gc.collect()

In [ ]:
NCHUNK = 10

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

wbin = np.linspace(0, 1701, int(1701 / 0.3) + 1)
wlo = wbin[:-1]
whi = wbin[1:]
ws = (wlo + whi) / 2.

itpc_bins = np.linspace(0, 4, 5) - 0.5

tpcs = ["EE", "EW", "WE", "WW"]


qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2. 

In [ ]:
# drivtV_file = datadir + "driftV_w.txt"
lifetime_file = datadir + "lifetimes_runArun1run2.txt"
scaleyz_file_RUN1 = datadir + "P1_scaleYZ_Run2.txt"
scaleyz_file_RUN2 = datadir + "P1_scaleYZ_Run2.txt"

In [ ]:
def process_file(inp):
    i, f = inp

    runname = f.split("/")[6]
    scaleyz_file = scaleyz_file_RUN2 if runname == "Run2" else scaleyz_file_RUN1
    
    # print("loading!")
    df = pd.read_hdf(f)
    for c in df.columns:
        if c.startswith("true") or c.startswith("dir"):
            del df[c]
    del df["sumadc"]
    del df["width"]
    
    # print("loaded!")
    
    df["tpcEE"] = df.tpcE & (df.cryostat == 0)
    df["tpcEW"] = df.tpcW & (df.cryostat == 0)
    df["tpcWE"] = df.tpcE & (df.cryostat == 1)
    df["tpcWW"] = df.tpcW & (df.cryostat == 1)

    df["dqdx"] = df.charge / df.pitch
    df["thitp"] = (df.time * tick_period - df.pandora_t0 - tanode*tick_period) / 1000.

    if "bnb" in f:
        df["bnb"] = True
    else:
        df["bnb"] = False
        
    df["ifile"] = i

    df["lifetime"] = np.nan

    fileruns = df.run.unique()
    if not isMC:
        with open(lifetime_file) as f:
            next(f)
            for line in f:
                dat = list(map(float, line.rstrip("\n").split(" ")))
                run = int(dat[0])
                
                if run not in fileruns: continue
                
                df.loc[(df.run == run) & df.tpcEE, "lifetime"] = dat[1]
                df.loc[(df.run == run) & df.tpcEW, "lifetime"] = dat[2]
                df.loc[(df.run == run) & df.tpcWE, "lifetime"] = dat[3]
                df.loc[(df.run == run) & df.tpcWW, "lifetime"] = dat[4]
    else:
        df.lifetime = 3.5e3
        
    # print("Made lifetimes!")

    df["itpc"] = -1
    df.loc[df.tpcEE, "itpc"] = 0
    df.loc[df.tpcEW, "itpc"] = 1
    df.loc[df.tpcWE, "itpc"] = 2
    df.loc[df.tpcWW, "itpc"] = 3

    df["scale_yz"] = np.nan

    if not isMC:

        df["ybin"] = np.searchsorted(yz_ybin, df.y.values) - 1
        df["zbin"] = np.searchsorted(yz_zbin, df.z.values) - 1

        idx = []
        scales = []

        with open(scaleyz_file) as f:
            next(f)
            for line in f:
                dat = line.rstrip("\n").split("\t")
                tpc = dat[0]
                iy = int(dat[1])
                iz = int(dat[2])
                scale = float(dat[3])

                idx.append((tpcnames.index(tpc), iy, iz))
                scales.append(scale)

        scaledf = pd.DataFrame(scales, index=pd.MultiIndex.from_tuples(idx, names=["itpc", "ybin", "zbin"]), 
                               columns=["scale_yz"])
        dtmp = pd.merge(df[["itpc", "ybin", "zbin"]], scaledf, on=["itpc", "ybin", "zbin"], how="left")

        df["scale_yz"] = dtmp.scale_yz.values

        del dtmp

    else:
        df.scale_yz = 1

    df["dqdx_normt"] = df.dqdx * np.exp(df.thitp / df.lifetime) / df.scale_yz
    when = (df.pitch > 0.3*NCHUNK) & (df.pitch < 1*NCHUNK) & (df.thitp > 50) & (df.thitp < 900) #&\

    Ndqdx_nocorr, _ = np.histogram(df.dqdx[when], 
                                   bins=qbins)
    Ndqdx_tcorr, _ = np.histogram((df.dqdx*np.exp(df.thitp / df.lifetime))[when], 
                                  bins=qbins)
    Ndqdx_wtcorr, _ = np.histogram((df.dqdx * np.exp(df.thitp / df.lifetime) / df.scale_yz)[when], 
                                   bins=qbins)
    return Ndqdx_nocorr, Ndqdx_tcorr, Ndqdx_wtcorr

In [ ]:
# process_file((0, files[0]))

In [ ]:
Nnocorr = []
Ntcorr = []
Nwtcorr = []

with Pool(processes=12) as pool:
    for nocorr, tcorr, wtcorr in tqdm(pool.imap_unordered(process_file, enumerate(files)), total=len(files)):
        Nnocorr.append(nocorr)
        Ntcorr.append(tcorr)
        Nwtcorr.append(wtcorr)

In [ ]:
qbins_centers = (qbins[:1] + qbins[:-1]) /2

In [ ]:
Ndqdx_nocorr = np.sum(Nnocorr, axis=0)

Ndqdx_tcorr = np.sum(Ntcorr, axis=0)

Ndqdx_wtcorr = np.sum(Nwtcorr, axis=0)

In [ ]:
_ = plt.hist(qbin_centers, bins=qbins, weights=Ndqdx_nocorr, histtype="step", 
             linewidth=2, label="No Correction")
_ = plt.hist(qbin_centers, bins=qbins, weights=Ndqdx_tcorr, histtype="step", 
             linewidth=2, label="Drift Equalization")
_ = plt.hist(qbin_centers, bins=qbins, weights=Ndqdx_wtcorr, histtype="step", 
             linewidth=2, label="Drift + Wire Plane\nEqualization")
plt.xlim([100, 2400])
plt.text(0.4, 0.35, "Run A + Run 1 + Run 2 Data\nThroughgoing Cosmic Muons\n3 < 10-Wire-Pitch < 10 cm\nMiddle Induction Plane",
        fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("10-Wire $dQ/dx$ [ADC/cm]")
plt.ylabel("# Depositions")
plt.legend()
plt.tight_layout()
if dosave: plt.savefig(savedir + "throughgoing_dqdx.pdf")